# Total test Passed

This notebook creates a Total Test passed report for test results. It ties into the **Test Monitor Service** for retrieving filtered test results, the **Notebook Execution Service** for running outside of Jupyterhub, and the **Test Monitor Reports page** at #testmonitor/reports for displaying results.

The parameters and output use a schema recognized by the Test Monitor Reports page, which can be implemented by various report types. The Total Test Passed notebook produces data that is best shown in a bar graph.

### Imports
Import Python modules for executing the notebook. Pandas is used for building and handling dataframes. Scrapbook is used for recording data for the Notebook Execution Service. The SystemLink Test Monitor Client provides access to test result data for processing.

In [1]:
import copy
import datetime
import pandas as pd
import scrapbook as sb
from dateutil import tz

import systemlink.clients.nitestmonitor as testmon

### Parameters
- `group_by`: The dimension along which to reduce; what each bar in the output graph represents  
  Options: Day, System, Test Program, Operator, Part Number  
  Default: Day
- `results_filter`: Dynamic Linq query filter for test results from the Test Monitor Service  
  Options: Any valid Test Monitor Results Dynamic Linq filter  
  Default: `'startedWithin <= "30.0:0:0"'`

Parameters are also listed in the metadata for the parameters cell, along with their default values. The Notebook Execution services uses that metadata to pass parameters from the Test Monitor Reports page to this notebook. Available `group_by` options are listed in the metadata as well; the Test Monitor Reports page uses these to validate inputs sent to the notebook.

To see the metadata, select the code cell and click the wrench icon in the far left panel.

In [2]:
results_filter = 'startedWithin <= "20.0:0:0" && workspacename = "DEV Manufacturing"'
products_filter = ''
group_by = 'System'

### Mapping from grouping options to Test Monitor terminology
Translate the grouping options shown in the Test Monitor Reports page to keywords recognized by the Test Monitor API.

In [3]:
groups_map = {
    'Day': 'started_at',
    'System': 'host_name',
    'Test Program': 'program_name',
    'Operator': 'operator',
    'Part Number': 'part_number',
    'Workspace': 'workspace'
}
grouping = groups_map[group_by]
display(grouping)

'host_name'

### Create Test Monitor client
Establish a connection to SystemLink over HTTP.

In [4]:
results_api = testmon.ResultsApi()

### Query for results
Query the Test Monitor Service for results matching the `results_filter` parameter.

In [5]:
results_query = testmon.ResultsAdvancedQuery(
    results_filter, product_filter=products_filter, order_by=testmon.ResultField.STARTED_AT, take=1000
)

results = []

response = await results_api.query_results_v2(post_body=results_query)

while response.continuation_token:
    results = results + response.results
    results_query.continuation_token = response.continuation_token
    response = await results_api.query_results_v2(post_body=results_query)

results_list = [result.to_dict() for result in results]
#display((results_list))

### Get group names
Collect the group name for each result based on the `group_by` parameter.

In [6]:
group_names = []
for result in results_list:
    if grouping != "host_name":
        if grouping in result:
            group_names.append(result[grouping])
    else:
        if 'properties' in result:
            group_names.append(result['properties']['Location'])    

### Create pandas dataframe
Put the data into a dataframe whose columns are test result id, status, and group name.

In [7]:
formatted_results = {
    'id': [result['id'] for result in results_list],
    'status': [result['status']['status_type'] if result['status'] else None for result in results_list],
    grouping: group_names
}

df_results = pd.DataFrame.from_dict(formatted_results)
#display(df_results)

### Handle grouping by day
If the grouping is by day, the group name is the date and time when the test started in UTC. To group all test results from a single day together, convert to server time and remove time information from the group name.

In [8]:
df_results_copy = copy.copy(df_results)
df_results_copy.fillna(value='', inplace=True)

if grouping == 'started_at':
    truncated_times = []
    for val in df_results_copy[grouping]:
        local_time = val.astimezone(tz.tzlocal())
        truncated_times.append(str(datetime.date(local_time.year, local_time.month, local_time.day)))
    df_results_copy[grouping] = truncated_times

### Aggregate results into groups
Aggregate the data for each unique group and status.

*See documentation for [size](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.size.html) and [unstack](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.unstack.html) here.*

In [9]:
df_grouped = df_results_copy.groupby([grouping, 'status']).size().unstack(fill_value=0)
if 'PASSED' not in df_grouped:
    df_grouped['PASSED'] = 0
if 'FAILED' not in df_grouped:
    df_grouped['FAILED'] = 0
if 'ERRORED' not in df_grouped:
    df_grouped['ERRORED'] = 0

### Total Passed
Divide the number of passed tests by the total number of tests. Terminated and timed out is not taken for calculation
df_pass_rate = pd.DataFrame(100 * df_grouped['PASSED'] / (df_grouped['FAILED'] + df_grouped['ERRORED'] + df_grouped['PASSED']))
df_pass_rate = df_pass_rate.reset_index().set_axis([grouping, 'pass_rate'], axis=1)

if grouping == 'started_at':
    df_pass_rate['started_at'] = pd.to_datetime(df_pass_rate['started_at'])
else:
    df_pass_rate.sort_values(by=['pass_rate'], ascending=True, inplace=True)

In [10]:
df_total_passed= pd.DataFrame(df_grouped['PASSED'] )
df_total_passed = df_total_passed.reset_index().set_axis([grouping, 'total_passed'], axis=1)

if grouping == 'started_at':
    df_total_passed['started_at'] = pd.to_datetime(df_total_passed['started_at'])
else:
    df_total_passed.sort_values(by=['total_passed'], ascending=True, inplace=True)
print(df_total_passed)

Empty DataFrame
Columns: [host_name, total_passed]
Index: []


### Convert the dataframe to the SystemLink reports output format
The result format for a SystemLink report consists of a list of output objects as defined below:
- `type`: The type of the output. Accepted values are 'data_frame' and 'scalar'.
- `id`: Corresponds to the id specified in the 'output' metadata. Used for returning multiple outputs with the 'V2' report format.
- `data`: A dict representing the 'data_frame' type output data.
    - `columns`: A list of dicts containing the names and data type for each column in the dataframe.
    - `values`: A list of lists containing the dataframe values. The sublists are ordered according to the 'columns' configuration.
- `value`: The value returned for the 'scalar' output type.
- `config`: The configurations for the given output.
    - `title`: The output title.
    - `graph`: The graph configurations.
        - `axis_labels`: The x-axis label and y-axis label.
        - `plots`: A list of plots to display mapped from the dataframe's columns, along with configuration options.
            - `x`: The dataframe column corresponding to the x-axis values.
            - `y`: The dataframe column corresponding to the y-axis values.
            - `style`: The plot's style. Accepted values are ['LINE', 'BAR', 'SCATTER'].
            - `color`: The plot's color. Accepted formats are ['blue', '#0000ff', 'rbg(0,0,255)'].
            - `label`: The plot's name, to be shown in a plot legend. 
            - `secondary_y`: Whether or not to display this plot on a second y-axis.
            - `group_by`: A list of columns in the dataframe on which to group data, e.g. to color individual points.
        - `orientation`: 'HORIZONTAL' or 'VERTICAL'.
        - `stacked`: Whether or not to display the plots stacked on top of each other.

Here is an example of a notebook result with two outputs, one of which is a dataframe with two columns, and the other is a scalar value:
```
[{
    'type': 'data_frame',
    'id': 'output_id_1',
    'data': {
        'columns': [
            {'name': 'time', 'type': 'datetime'},
            {'name': 'value', 'type': 'number'}
         ],
        'values': [
            ['2020-09-29T00:00:00.000Z', 46.1538461538],
            ['2020-09-30T00:00:00.000Z', 63.1578947368],
            ...
         ]
    },
    'config': {
        'title': 'My Title',
        'graph': {
            'axis_labels': ['X Axis', 'Y Axis'],
            'orientation': 'VERTICAL',
            'plots': [
                {'x': 'time', 'y': 'value', 'style': 'BAR', 'color': '#0000ff', 'label': 'Plot 1'}
            ]
        }
    }
}, {
    'type': 'scalar',
    'id': 'output_id_2',
    'config': {
        'title': 'My Title'
    },
    'value': 5
}]
```

For this report, there is one output, which is a dataframe with two columns. For a grouping of 'Day', the first column contains ISO-8601 date strings. For any other grouping option, the first column contains categorical string values. The second column contains numerical values representing the pass rate as a percentage.

| started_at                 | pass_rate     |
|----------------------------|---------------|
| '2020-09-29'               | 46.1538461538 |
| '2020-09-30'               | 63.1578947368 |
| '2020-10-01'               | 60.0          |

The graph configuration specifies a single plot, where the x-axis is the group values and the y-axis is the pass rate. We use Pandas to convert the dataframe built in the previous cells into a tabular format and then return that with the result object.

In [11]:
df_dict = {
    'columns': pd.io.json.build_table_schema(df_total_passed, index=False)['fields'],
    'values': df_total_passed.values.tolist(),
}

total_passed_graph = {
    'type': 'data_frame',
    'id': 'total_passed_graph',
    'data': df_dict,
    'config': {
        'title': 'Total Passed by {}'.format(group_by),
        'graph': {
            'axis_labels': [group_by, 'Total Passed'],
            'plots': [
                {'x': grouping, 'y': 'total_passed', 'style': 'BAR'}
            ]
        }
    }
}

if grouping == 'started_at':
    total_passed_graph['config']['graph']['orientation'] = 'VERTICAL'
else:
    total_passed_graph['config']['graph']['orientation'] = 'HORIZONTAL'
    total_passed_graph['config']['graph']['plots'][0]['group_by'] = [grouping]

result = [total_passed_graph]
print(result)

[{'type': 'data_frame', 'id': 'total_passed_graph', 'data': {'columns': [{'name': 'host_name', 'type': 'number'}, {'name': 'total_passed', 'type': 'integer'}], 'values': []}, 'config': {'title': 'Total Passed by System', 'graph': {'axis_labels': ['System', 'Total Passed'], 'plots': [{'x': 'host_name', 'y': 'total_passed', 'style': 'BAR', 'group_by': ['host_name']}], 'orientation': 'HORIZONTAL'}}}]


### Record results with Scrapbook

In [12]:
sb.glue('result', result)